# 3. Transformer Modeling

### Upload sample of 5k lyrics generated in 1st notebook

In [1]:
from google.colab import files
import io
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')



lyrics_df = pd.read_csv('/content/drive/MyDrive/w266_final_project/source/lyrics_df_5k.csv')
lyrics_df.head()

Mounted at /content/drive


,track_id,track_name,track_artist,valence,lyrics_snippet,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,tempo,duration_ms
0,2XU0oxnq2qxCpomAAuJY8K,Dance Monkey,Tones and I,0.513,"They say, ""Oh my god, I see the way you shine ...",100,0UywfDKYlyiu1b38DRrzYD,Dance Monkey (Stripped Back) / Dance Monkey,2019-10-17,"post-teen alternative, indie, pop (large variety)",...,0.588,6,-6.4,0,0.0924,0.692,0.000104,0.149,98.027,209438
1,2XU0oxnq2qxCpomAAuJY8K,Dance Monkey,Tones and I,0.513,"They say, ""Oh my god, I see the way you shine ...",100,0UywfDKYlyiu1b38DRrzYD,Dance Monkey (Stripped Back) / Dance Monkey,2019-10-17,"post-teen alternative, indie, pop (large variety)",...,0.588,6,-6.4,0,0.0924,0.692,0.000104,0.149,98.027,209438
2,2XU0oxnq2qxCpomAAuJY8K,Dance Monkey,Tones and I,0.513,"They say, ""Oh my god, I see the way you shine ...",100,0UywfDKYlyiu1b38DRrzYD,Dance Monkey (Stripped Back) / Dance Monkey,2019-10-17,"post-teen alternative, indie, pop (large variety)",...,0.588,6,-6.4,0,0.0924,0.692,0.000104,0.149,98.027,209438
3,2XU0oxnq2qxCpomAAuJY8K,Dance Monkey,Tones and I,0.513,"They say, ""Oh my god, I see the way you shine ...",100,0UywfDKYlyiu1b38DRrzYD,Dance Monkey (Stripped Back) / Dance Monkey,2019-10-17,"post-teen alternative, indie, pop (large variety)",...,0.588,6,-6.4,0,0.0924,0.692,0.000104,0.149,98.027,209438
4,2XU0oxnq2qxCpomAAuJY8K,Dance Monkey,Tones and I,0.513,"They say, ""Oh my god, I see the way you shine ...",100,0UywfDKYlyiu1b38DRrzYD,Dance Monkey (Stripped Back) / Dance Monkey,2019-10-17,"post-teen alternative, indie, pop (large variety)",...,0.588,6,-6.4,0,0.0924,0.692,0.000104,0.149,98.027,209438


In [15]:
print(f"# unique songs: {lyrics_df.track_id.nunique()}")
print(f"# rows in df: {lyrics_df.shape[0]}")

## looks like some duplicate rows came in through the import - this is caused by playlist-level redundancies

df = lyrics_df.drop(columns = ['playlist_name', 'playlist_id',
       'playlist_genre', 'playlist_subgenre', 'track_id']).drop_duplicates()

## drop null/na lyric snippets

df = df.dropna(subset = ['lyrics_snippet'])

print(f"# unique songs (post-dedupe): {df.index.nunique()}")
print(f"# rows in df (post-dedupe): {df.shape[0]}")


# unique songs: 5291
# rows in df: 1550070
# unique songs (post-dedupe): 4683
# rows in df (post-dedupe): 4683


### df cleaning: removing non-english lyrics

In [19]:
import sys
!{sys.executable} -m pip install langdetect

In [20]:
from langdetect import detect, DetectorFactory
import numpy as np

# Ensure consistent results from langdetect
DetectorFactory.seed = 0

def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False # Return False for any detection errors (e.g., very short or empty strings)

print(f"Initial DataFrame shape: {df.shape}")

# Apply the language detection function
df['is_english'] = df['lyrics_snippet'].apply(is_english)

# Filter out non-English snippets
df_english = df[df['is_english']].drop(columns=['is_english'])

print(f"DataFrame shape after dropping non-English lyrics: {df_english.shape}")

df = df_english # Update the main df to contain only English lyrics

Initial DataFrame shape: (4683, 19)
DataFrame shape after dropping non-English lyrics: (3717, 19)


#### With an operable, deduped and cleaned dataframe, we are ready to conduct our analysis using a subset of 3,717 unique tracks.

In [22]:
import pandas as pd

df.to_csv('/content/drive/MyDrive/w266_final_project/source/lyrics_df_cleaned.csv')